In [7]:
import pickle
import numpy as np
import msmrd2
from msmrd2.markovModels import continuousTimeMarkovStateModel as ctmsm
from msmrd2.markovModels import msmrdMarkovModel as msmrdMSM
from msmrd2.integrators import msmrdIntegrator 

In [1]:
# Main parameters
numBoundStates = 8
maxNumBoundStates = 10
radialBounds = [1.25, 2.25]
relativeDistanceCutOff = radialBounds[1]
numParticleTypes = 1 # num. of particle types (not states) in unbound state
boxsize = 6

# Discretization parameters (need to be consistent with those of integrator
numSphericalSectionsPos = 7 #7
numRadialSectionsQuat = 5 #5
numSphericalSectionsQuat = 7 #7
totalnumSecsQuat = numSphericalSectionsQuat*(numRadialSectionsQuat -1) + 1
numTransitionsStates = numSphericalSectionsPos * totalnumSecsQuat #203 

# Trajectory relevant parameters (Need to be equal to the one used to generate the rateDictionary)
dt = 0.0001
original_dt = 0.00001
lagtime = 150
timesteps = 3000000 # 2000000
stride = 25 #2500
realLagtime = lagtime*original_dt*stride
maxentries = int(timesteps/stride)

In [9]:
# Load pickled rateDicitionary generated in generateRateDictionary
pickle_in = open("../data/pickled_data/MSM_dimer_t3.00E+06_s25_lagt150.pickle","rb")
pickle_in = open("../data/pickled_data/MSM_patchyProtein_t6.00E+06_s50_lagt75.pickle","rb")
mainMSM = pickle.load(pickle_in)
tmatrix = mainMSM['transition_matrix']
activeSet = mainMSM['active_set']

In [10]:
activeSet

array([  1,   2,   3,   4,   5,   6,  11,  12,  13,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
       109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
       122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
       135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
       148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160,
       161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173,
       174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 18

In [ ]:
# Define continuous-time MSM for unbound dynamics (same for all particles)
MSMtype = 0
ratematrix = np.array([[0]]) # no unbound dynamics
seed = 0 # Seed = -1 used random device as seed
unboundMSM = ctmsm(MSMtype, ratematrix, seed)
Dlist = np.array([1.0])
Drotlist = np.array([1.0])
unboundMSM.setD(Dlist)
unboundMSM.setDrot(Drotlist)

In [ ]:
# Define coupling Markov model
seed = 0
Dbound = [0.5]*numBoundStates
DboundRot = [1.0]*numBoundStates
couplingMSM = msmrdMSM(numBoundStates, maxNumBoundStates, tmatrix, activeSet, realLagtime, seed) 
couplingMSM.setDbound(Dbound, DboundRot)

In [ ]:
# Define particle list (make sure all particles are inside box)
ptype = 0
pstate = 0
D = Dlist[pstate]
Drot = Drotlist[pstate]
position = np.array([1,1,1])
orientation = np.array([1,0,0,0])
part1 = msmrd2.particle(ptype, pstate, D, Drot, 0.1*position, orientation)
part2 = msmrd2.particle(ptype, pstate, D, Drot, -0.1*position, orientation)
part1.deactivateMSM()
part2.deactivateMSM()
partList = msmrd2.integrators.particleList([part1, part2])

In [ ]:
# MSM/RD integrator definition
dt = 0.002 # should be smaller than Gillespie inverse transition rates
seed = 0 # Seed = -1 used random device as seed
bodytype = 'rigidbody'
integrator = msmrdIntegrator(dt, seed, bodytype, numParticleTypes, radialBounds, unboundMSM, couplingMSM)

In [ ]:
# Set boundary and discretization in integrator (optional since default values also work)
boundary = msmrd2.box(boxsize,boxsize,boxsize,'periodic')
integrator.setBoundary(boundary)
discretization = msmrd2.discretizations.positionOrientationPartition(relativeDistanceCutOff, numSphericalSectionsPos,
                                                                    numRadialSectionsQuat, numSphericalSectionsQuat)
integrator.setDiscretization(discretization)

In [ ]:
print(partList[0].position, partList[1].position, 
      partList[0].state, partList[1].state, 
      partList[0].boundTo, partList[1].boundTo)
prevstate = 1

In [ ]:
prevstate = 0
for i in range(20000000):
    integrator.integrate(partList)
    #print(partList[0].position, partList[1].position, partList[0].state, partList[1].state, partList[0].boundTo, partList[1].boundTo)
    if (partList[0].boundState != prevstate):
        print("{:.4f}".format(integrator.clock), partList[0].boundState, partList[1].boundState)
        #print(partList[0].position, partList[1].position, partList[0].state, partList[1].state, partList[0].boundTo, partList[1].boundTo)
        prevstate = partList[0].boundState